In [1]:
cd ..

/home/mehdi2277/Documents/HarveyMuddWork/Neural_Nets_Research/neural_nets_research


In [2]:
import torch.optim as optim
import torch.nn as nn
import torch
import matplotlib.pyplot as plt

from neural_nets_library import training
from tree_to_sequence.tree_encoder import TreeEncoder
from tree_to_sequence.tree_decoder import TreeDecoder
# from tree_to_sequence.grammar_tree_decoder import GrammarTreeDecoder
from tree_to_sequence.program_datasets import IdentityTreeToTreeDataset, ForLambdaDataset, Const0
from tree_to_sequence.translating_trees import ( parent_to_category_FOR, category_to_child_FOR,
                                                 category_to_child_LAMBDA, 
                                                 parent_to_category_LAMBDA, ForGrammar, For,
                                                 LambdaGrammar, Lambda)
from tree_to_sequence.tree_to_tree_attention import TreeToTreeAttention
from functools import partial

In [3]:
num_vars = 10
num_ints = 11

for_ops = {
    "Var": 0,
    "Const": 1,
    "Plus": 2,
    "Minus": 3,
    "EqualFor": 4,
    "LeFor": 5,
    "GeFor": 6,
    "Assign": 7,
    "If": 8,
    "Seq": 9,
    "For": 10
}

for_ops = {"<" + k.upper() + ">": v for k,v in for_ops.items()}

lambda_calculus_ops = {
                "<VARIABLE>": 0,
                "<ABSTRACTION>": 1,
                "<NUMBER>": 2,
                "<BOOLEAN>": 3,
                "<NIL>": 4,
                "<IF>": 5,
                "<CONS>": 6,
                "<MATCH>": 7,
                "<UNARYOPER>": 8,
                "<BINARYOPER>": 9,
                "<LET>": 10,
                "<LETREC>": 11,
                "<TRUE>": 12,
                "<FALSE>": 13,
                "<TINT>": 14,
                "<TBOOL>": 15,
                "<TINTLIST>": 16,
                "<TFUN>": 17,
                "<ARGUMENT>": 18,
                "<NEG>": 19,
                "<NOT>": 20,
                "<PLUS>": 21,
                "<MINUS>": 22,
                "<TIMES>": 23,
                "<DIVIDE>": 24,
                "<AND>": 25,
                "<OR>": 26,
                "<EQUAL>": 27,
                "<LESS>": 28,
                "<APPLICATION>": 29,
                "<HEAD>": 30,
                "<TAIL>": 31
            }

lambda_ops = {
            "<VAR>": 0,
            "<CONST>": 1,
            "<PLUS>": 2,
            "<MINUS>": 3,
            "<EQUAL>": 4,
            "<LE>": 5,
            "<GE>": 6,
            "<IF>": 7,
            "<LET>": 8,
            "<UNIT>": 9,
            "<LETREC>": 10,
            "<APP>": 11,
        }

In [4]:
use_embedding = True
binarize = True
eos_tokens = True
long_base_case = True

In [5]:
# identity_dset = IdentityTreeToTreeDataset("ANC/Easy-arbitraryForListWithOutput.json", 
#                                       binarize=binarize, is_lambda_calculus=False, 
#                                       num_ints=num_ints, num_vars=num_vars,
#                                       use_embedding=use_embedding, cuda=False)
for_lambda_dset = ForLambdaDataset("ANC/VeryHard-arbitraryForList.json", binarize=binarize, 
                                   eos_tokens=eos_tokens, use_embedding=use_embedding, 
                                   long_base_case=long_base_case, input_as_seq=False, output_as_seq=False)
print(len(for_lambda_dset))
# Calculate max-size so we're sure it's good enough
max_size = max([pair[1].size() for pair in for_lambda_dset])
print(max_size)

50000
245


In [6]:
def reset_all_parameters_uniform(model, stdev):
    for param in model.parameters():
        nn.init.uniform(param, -stdev, stdev)

In [7]:
embedding_size = 100 #... 256 is from the paper, but 100 is WAY faster
hidden_size = 256
num_layers = 1
alignment_size = 50
align_type = 1
encoder_input_size = num_vars + num_ints + len(for_ops) + 1
encoder = TreeEncoder(encoder_input_size, hidden_size, num_layers, [1, 2, 3, 4, 5], attention=True, use_embedding=use_embedding)
nclass = num_vars + num_ints + len(lambda_ops)
plot_every = 100
max_num_children = 4

decoder = TreeDecoder(embedding_size, hidden_size, max_num_children, nclass=nclass)
# decoder = GrammarTreeDecoder(embedding_size, hidden_size, max_num_children, 
#                              parent_to_category_LAMBDA, len(LambdaGrammar), category_to_child_LAMBDA,
#                             num_vars, num_ints, binarized=binarize)

program_model = TreeToTreeAttention(encoder, decoder, hidden_size, embedding_size, nclass=nclass,
                           alignment_size=alignment_size, align_type=align_type, max_size=max_size)
    
reset_all_parameters_uniform(program_model, 0.1)
encoder.initialize_forget_bias(3)
decoder.initialize_forget_bias(3)

In [8]:
# program_model = program_model.cuda()

In [9]:
optimizer = optim.Adam(program_model.parameters(), lr=0.005)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=500, factor=0.8)



In [10]:
# Token accuracy of the program
def count_matches(prediction, target):
    matches = 0
    if int(prediction.value) == int(target.value):
        matches = matches + 1
    for i in range(min(len(target.children), len(prediction.children))):
        matches = matches + count_matches(prediction.children[i], target.children[i])
    size_diff = abs(len(target.children) - len(prediction.children))
    return matches - size_diff


# Program accuracy (1 if completely correct, 0 otherwise)
def all_matches(prediction, target):
    # Nodes must have the same value
    x = target.value
    if target in range(num_ints):
        x = x % 4
    if not int(prediction.value) == int(x): #TODO - change this back! And change def vectorize
        return 0
    # ... and the same number of kids
    if not len(prediction.children) == len(target.children):
        return 0
    n = len(prediction.children)
    # ... and match perfectly for each kid
    for i in range(n):
        if all_matches(prediction.children[i], target.children[i]) == 0:
            return 0
    return 1

# Calc validation accuracy (this could either be program or token accuracy)
def validation_criterion(prediction, target):
    return all_matches(prediction, target)

In [ ]:

best_model, train_plot_losses, validation_plot_losses = training.train_model_tree_to_tree(program_model, for_lambda_dset, 
                                 optimizer, lr_scheduler=None, num_epochs=1, plot_every=plot_every,
                                 batch_size=100, print_every=200, validation_criterion=validation_criterion,
                                 use_cuda=False) #TODO:  epochs=3



Epoch 0/0
----------
Epoch Number: 0, Batch Number: 200, Training Loss: 219.8732
Time so far is 0m 47s
Example diff:
DESIRED
   ┌33
 29┤
   │     ┌33
   │  ┌12┤
   │  │  └33
   └21┤
      │     ┌33
      │  ┌10┤
      │  │  └33
      └22┤
         │  ┌33
         └29┤
            │  ┌33
            └19┤
               │     ┌33
               │  ┌29┤
               │  │  │     ┌33
               │  │  │  ┌12┤
               │  │  │  │  └33
               │  │  └21┤
               │  │     │     ┌33
               │  │     │  ┌12┤
               │  │     │  │  └33
               │  │     └21┤
               │  │        │  ┌33
               │  │        └30┤
               │  │           └33
               └29┤
                  │  ┌33
                  └19┤
                     │     ┌33
                     │  ┌29┤
                     │  │  │     ┌33
                     │  │  │  ┌12┤
                     │  │  │  │  └33
                     │  │  └21┤
                     │  │     │ 

Epoch Number: 0, Batch Number: 1000, Training Loss: 116.0710
Time so far is 3m 30s
Example diff:
DESIRED
   ┌33
 28┤
   │       ┌33
   │     ┌1┤
   │     │ └33
   │  ┌22┤
   │  │  │  ┌33
   │  │  └22┤
   │  │     │ ┌33
   │  │     └5┤
   │  │       └33
   └26┤
      │     ┌33
      │  ┌29┤
      │  │  │     ┌33
      │  │  │  ┌12┤
      │  │  │  │  └33
      │  │  └21┤
      │  │     │    ┌33
      │  │     │  ┌3┤
      │  │     │  │ └33
      │  │     └22┤
      │  │        │  ┌33
      │  │        └30┤
      │  │           └33
      └29┤
         │     ┌33
         │  ┌12┤
         │  │  └33
         └21┤
            │     ┌33
            │  ┌30┤
            │  │  └33
            └23┤
               │    ┌33
               │  ┌4┤
               │  │ └33
               └22┤
                  │  ┌33
                  └22┤
                     │ ┌33
                     └3┤
                       └33
WE GOT!
   ┌33
 29┤
   └33
Epoch Number: 0, Batch Number: 1000, Validation Metric: 0.00

Epoch Number: 0, Batch Number: 2000, Training Loss: 97.6534
Time so far is 6m 37s
Example diff:
DESIRED
   ┌33
 29┤
   │     ┌33
   │  ┌12┤
   │  │  └33
   └21┤
      │     ┌33
      │  ┌30┤
      │  │  └33
      └23┤
         │    ┌33
         │  ┌2┤
         │  │ └33
         └22┤
            │  ┌33
            └22┤
               │  ┌33
               └10┤
                  └33
WE GOT!
   ┌33
 29┤
   │  ┌33
   └21┤
      └33
Epoch Number: 0, Batch Number: 2000, Validation Metric: 0.0000
Epoch Number: 0, Batch Number: 2200, Training Loss: 87.6790
Time so far is 7m 12s
Example diff:
DESIRED
   ┌33
 31┤
   │  ┌33
   └20┤
      │  ┌33
      └12┤
         │     ┌33
         │  ┌32┤
         │  │  │  ┌33
         │  │  └20┤
         │  │     │  ┌33
         │  │     └22┤
         │  │        │ ┌33
         │  │        └4┤
         │  │          └33
         └28┤
            │       ┌33
            │     ┌6┤
            │     │ └33
            │  ┌22┤
            │  │  │  ┌33
            │

Epoch Number: 0, Batch Number: 3200, Training Loss: 69.1372
Time so far is 10m 25s
Example diff:
DESIRED
   ┌33
 29┤
   │     ┌33
   │  ┌12┤
   │  │  └33
   └21┤
      │    ┌33
      │  ┌7┤
      │  │ └33
      └22┤
         │  ┌33
         └30┤
            └33
WE GOT!
   ┌33
 29┤
   │  ┌33
   └21┤
      │  ┌33
      └22┤
         └33
Epoch Number: 0, Batch Number: 3200, Validation Metric: 0.0000
Epoch Number: 0, Batch Number: 3400, Training Loss: 64.9817
Time so far is 11m 4s
Example diff:
DESIRED
   ┌33
 29┤
   │     ┌33
   │  ┌12┤
   │  │  └33
   └21┤
      │    ┌33
      │  ┌3┤
      │  │ └33
      └22┤
         │  ┌33
         └30┤
            └33
WE GOT!
   ┌33
 29┤
   │  ┌33
   └21┤
      │  ┌33
      └22┤
         └33
Epoch Number: 0, Batch Number: 3400, Validation Metric: 0.0000
Epoch Number: 0, Batch Number: 3600, Training Loss: 64.4664
Time so far is 11m 43s
Example diff:
DESIRED
   ┌33
 31┤
   │  ┌33
   └20┤
      │  ┌33
      └12┤
         │     ┌33
         │  ┌32┤
      

Epoch Number: 0, Batch Number: 4800, Training Loss: 36.4623
Time so far is 15m 42s
Example diff:
DESIRED
   ┌33
 31┤
   │  ┌33
   └20┤
      │  ┌33
      └12┤
         │     ┌33
         │  ┌32┤
         │  │  │  ┌33
         │  │  └20┤
         │  │     │  ┌33
         │  │     └22┤
         │  │        │ ┌33
         │  │        └7┤
         │  │          └33
         └28┤
            │       ┌33
            │     ┌4┤
            │     │ └33
            │  ┌22┤
            │  │  │  ┌33
            │  │  └21┤
            │  │     │  ┌33
            │  │     └12┤
            │  │        └33
            └27┤
               │     ┌33
               │  ┌30┤
               │  │  └33
               └29┤
                  │  ┌33
                  └19┤
                     │     ┌33
                     │  ┌32┤
                     │  │  │  ┌33
                     │  │  └20┤
                     │  │     │  ┌33
                     │  │     └24┤
                     │  │        │     ┌33
   

...

Epoch Number: 0, Batch Number: 28600, Training Loss: 18.0891
Time so far is 99m 16s
Example diff:
DESIRED
   ┌33
 28┤
   │       ┌33
   │     ┌0┤
   │     │ └33
   │  ┌22┤
   │  │  │  ┌33
   │  │  └22┤
   │  │     │ ┌33
   │  │     └7┤
   │  │       └33
   └25┤
      │        ┌33
      │     ┌12┤
      │     │  └33
      │  ┌21┤
      │  │  │     ┌33
      │  │  │  ┌30┤
      │  │  │  │  └33
      │  │  └24┤
      │  │     │    ┌33
      │  │     │  ┌3┤
      │  │     │  │ └33
      │  │     └22┤
      │  │        │  ┌33
      │  │        └22┤
      │  │           │ ┌33
      │  │           └9┤
      │  │             └33
      └29┤
         │  ┌33
         └28┤
            │       ┌33
            │     ┌8┤
            │     │ └33
            │  ┌22┤
            │  │  │  ┌33
            │  │  └22┤
            │  │     │ ┌33
            │  │     └1┤
            │  │       └33
            └26┤
               │        ┌33
               │     ┌12┤
               │     │  └33
              

Epoch Number: 0, Batch Number: 29200, Training Loss: 15.8586
Time so far is 101m 21s
Example diff:
DESIRED
   ┌33
 29┤
   │     ┌33
   │  ┌12┤
   │  │  └33
   └21┤
      │     ┌33
      │  ┌30┤
      │  │  └33
      └24┤
         │    ┌33
         │  ┌5┤
         │  │ └33
         └22┤
            │  ┌33
            └22┤
               │ ┌33
               └5┤
                 └33
WE GOT!
   ┌33
 29┤
   │     ┌33
   │  ┌12┤
   │  │  └33
   └21┤
      │     ┌33
      │  ┌30┤
      │  │  └33
      └24┤
         │    ┌33
         │  ┌2┤
         │  │ └33
         └22┤
            │  ┌33
            └22┤
               │ ┌33
               └2┤
                 └33
Epoch Number: 0, Batch Number: 29200, Validation Metric: 0.0150
Epoch Number: 0, Batch Number: 29400, Training Loss: 17.9976
Time so far is 102m 5s
Example diff:
DESIRED
   ┌33
 28┤
   │       ┌33
   │     ┌8┤
   │     │ └33
   │  ┌22┤
   │  │  │  ┌33
   │  │  └22┤
   │  │     │ ┌33
   │  │     └5┤
   │  │       └33
   └26┤
     

Epoch Number: 0, Batch Number: 29800, Training Loss: 16.3678
Time so far is 103m 28s
Example diff:
DESIRED
   ┌33
 31┤
   │  ┌33
   └20┤
      │  ┌33
      └12┤
         │     ┌33
         │  ┌32┤
         │  │  │  ┌33
         │  │  └20┤
         │  │     │  ┌33
         │  │     └22┤
         │  │        │ ┌33
         │  │        └0┤
         │  │          └33
         └28┤
            │        ┌33
            │     ┌12┤
            │     │  └33
            │  ┌21┤
            │  │  │  ┌33
            │  │  └22┤
            │  │     │ ┌33
            │  │     └7┤
            │  │       └33
            └26┤
               │     ┌33
               │  ┌30┤
               │  │  └33
               └29┤
                  │  ┌33
                  └19┤
                     │     ┌33
                     │  ┌32┤
                     │  │  │  ┌33
                     │  │  └20┤
                     │  │     │  ┌33
                     │  │     └21┤
                     │  │        │  ┌33
    

Epoch Number: 0, Batch Number: 30400, Training Loss: 20.1890
Time so far is 105m 38s
Example diff:
DESIRED
   ┌33
 29┤
   │     ┌33
   │  ┌12┤
   │  │  └33
   └21┤
      │       ┌33
      │     ┌4┤
      │     │ └33
      │  ┌22┤
      │  │  │  ┌33
      │  │  └22┤
      │  │     │ ┌33
      │  │     └4┤
      │  │       └33
      └23┤
         │  ┌33
         └28┤
            │        ┌33
            │     ┌12┤
            │     │  └33
            │  ┌21┤
            │  │  │  ┌33
            │  │  └22┤
            │  │     │ ┌33
            │  │     └9┤
            │  │       └33
            └26┤
               │        ┌33
               │     ┌12┤
               │     │  └33
               │  ┌21┤
               │  │  │     ┌33
               │  │  │  ┌30┤
               │  │  │  │  └33
               │  │  └24┤
               │  │     │    ┌33
               │  │     │  ┌0┤
               │  │     │  │ └33
               │  │     └22┤
               │  │        │  ┌33
             

Epoch Number: 0, Batch Number: 30800, Training Loss: 17.5415
Time so far is 107m 4s
Example diff:
DESIRED
   ┌33
 29┤
   │     ┌33
   │  ┌12┤
   │  │  └33
   └21┤
      │       ┌33
      │     ┌5┤
      │     │ └33
      │  ┌22┤
      │  │  │  ┌33
      │  │  └22┤
      │  │     │ ┌33
      │  │     └1┤
      │  │       └33
      └23┤
         │  ┌33
         └29┤
            │     ┌33
            │  ┌12┤
            │  │  └33
            └21┤
               │    ┌33
               │  ┌4┤
               │  │ └33
               └22┤
                  │  ┌33
                  └29┤
                     │     ┌33
                     │  ┌12┤
                     │  │  └33
                     └21┤
                        │     ┌33
                        │  ┌30┤
                        │  │  └33
                        └24┤
                           │     ┌33
                           │  ┌12┤
                           │  │  └33
                           └21┤
                           

In [ ]:
train_plot_losses_old = train_plot_losses
validation_plot_losses_old = validation_plot_losses

In [ ]:
#%debug
plt.plot([x * plot_every for x in range(len(train_plot_losses_old + train_plot_losses))], train_plot_losses_old + train_plot_losses)
plt.show()

plt.plot([x * plot_every for x in range(len(validation_plot_losses_old + validation_plot_losses))], validation_plot_losses_old + validation_plot_losses)
plt.show()

In [ ]:
n = num_ints + num_vars

# Check whether a node is syntactically valid, given its parent and index
# Then recursively do it for all the node's children
def check_valid(node, parent, child_index):
    category = parent_to_category_LAMBDA(parent, child_index, num_vars, num_ints)
    possible_outputs = category_to_child_LAMBDA(category, num_vars, num_ints)
    if not int(node.value) in possible_outputs:
        print("parent", parent, "child_index", child_index)
        print("ERROR", int(node.value), category)
        return False
    if (len(node.children) > 0):
        child1 = check_valid(node.children[0], int(node.value), 0)
        if not child1:
            return False
        child2 = check_valid(node.children[1], parent, child_index + 1)
        if not child2:
            return False
    return True

# Check all the programs in a dataset for syntactic accuracy
# (this is a debugging function used to double check the accuracy of your grammar)
def check_all():
    i = 0
    # Check grammar is right
    for prog in for_lambda_dset:
        correct = check_valid(prog[1], None, 0)
        if correct is False:
            print(i)
            pretty_print_tree(prog[1])
            return
        i += 1
        
        
        
check_all() #kangaroo